In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout, Input
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os

2026-02-22 02:30:22.223619: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-22 02:30:22.223901: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-22 02:30:22.280111: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-22 02:30:23.748483: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

In [3]:
# --- 1. Load the Processed Data ---
print("Loading processed data...")
# Make sure the paths match where you saved them in the previous step
X_dcnn = np.load('../data/processed/X_dcnn.npy')
y_labels = np.load('../data/processed/y_labels.npy')

num_features = X_dcnn.shape[1]
print(f"Data loaded successfully. Input shape: {X_dcnn.shape}")

Loading processed data...
Data loaded successfully. Input shape: (1113112, 78, 1)


In [5]:
# --- 2. Train-Test Split ---
# We use stratify=y_labels to ensure both training and testing sets 
# have the same proportion of BENIGN vs. Malicious traffic.
X_train, X_test, y_train, y_test = train_test_split(
    X_dcnn, y_labels, test_size=0.2, random_state=42, stratify=y_labels
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

Training samples: 890489
Testing samples: 222623


In [6]:
# --- 3. Build the DCNN Architecture ---
print("Building the DCNN model...")
model = Sequential()

# Input Layer
model.add(Input(shape=(num_features, 1)))

# First Convolutional Layer
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))

# Second Convolutional Layer
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))

# Flatten the spatial dimensions to 1D
model.add(Flatten())

# Fully Connected Hidden Layer with L1/L2 Regularization and Dropout
model.add(Dense(
    units=256, 
    activation='relu', 
    kernel_regularizer=l1_l2(l1=0.01, l2=0.01)
))
model.add(Dropout(0.1))

# Output Layer
# Since you have binary labels (0 and 1), we use 2 units and softmax 
# to match the paper's categorical cross-entropy approach.
model.add(Dense(units=2, activation='softmax'))

Building the DCNN model...


2026-02-22 02:31:33.829707: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [7]:
# --- 4. Compile the Model ---
optimizer = Adam(learning_rate=0.0001)

# We use 'sparse_categorical_crossentropy' because your y_labels are integers (0, 1)
# rather than explicitly one-hot encoded arrays like [1, 0] or [0, 1].
model.compile(
    optimizer=optimizer, 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 76, 128)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 74, 256)        │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18944)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     4,849,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,949,506 (18.88 MB)

 Trainable params: 4,949,506 (18.88 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# --- 5. Train the Model ---
print("Starting training...")
history = model.fit(
    X_train, y_train,
    epochs=30,             
    batch_size=256,        
    validation_data=(X_test, y_test),
    verbose=1
)

Starting training...
Epoch 1/30
  15/3479 ━━━━━━━━━━━━━━━━━━━━ 8:55 155ms/step - accuracy: 0.6394 - loss: 5.6985